In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt


# Build Model

#### THIS CELL IS NOT RAN #####
##### Pre constructed this model and saved weights in "~/scratch/gpfs/eysu/SoftCL/models/ResNet50+CIFAR" ####

####################################################
## Build Model using ResNet50 and Classifier Head ##
####################################################

'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

In [ ]:
######################
## Load saved model ##
######################

model = tf.keras.models.load_model("~/scratch/gpfs/eysu/SoftCL/models/ResNet50+CIFAR_2")
model.summary()

# Load CIFAR10 Dataset

In [ ]:
##############################################
## Load, partition, and resize CIFAR10 Data ##
##############################################
def loadData():
    import pickle

    # unpickle the binary files
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    labels = ['airplane',  # index 0
          'automobile',  # index 1
          'bird',  # index 2 
          'cat',  # index 3 
          'deer',  # index 4
          'dog',  # index 5
          'frog',  # index 6 
          'horse',  # index 7 
          'ship',  # index 8 
          'truck']  # index 9
    
    # paths to each batch of data
    batch1 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_1")
    batch2 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_2")
    batch3 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_3")
    batch4 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_4")
    batch5 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_5")
    meta = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/batches.meta")
    test = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/test_batch")

    # separate labels and image data from each batch
    y_train1 = batch1[b'labels']
    x_train1 = batch1[b'data']
    y_train2 = batch2[b'labels']
    x_train2 = batch2[b'data']
    y_train3 = batch3[b'labels']
    x_train3 = batch3[b'data']
    y_train4 = batch4[b'labels']
    x_train4 = batch4[b'data']
    y_train5 = batch5[b'labels']
    x_train5 = batch5[b'data']

    # concatenate into big training and testing arrays
    y_train = np.concatenate((y_train1, y_train2, y_train3, y_train4, y_train5))
    x_train = np.concatenate((x_train1, x_train2, x_train3, x_train4, x_train5), axis=0)
    
    y_test = test[b'labels']
    x_test = test[b'data']
    
    # Further break training data into train / validation sets 
    # put 5000 into validation set and keep remaining 45,000 for train
    (x_train, x_valid) = x_train[1000:], x_train[:1000] 
    (y_train, y_valid) = y_train[1000:], y_train[:1000]

    # reshape data to match dimensions of cifar10.load_data
    x_train = x_train.reshape(49000, 3, 32, 32)
    x_train = x_train.transpose(0, 2, 3, 1)
    x_train = x_train.astype('float32')
    x_train /= 255

    x_valid = x_valid.reshape(1000, 3, 32, 32)
    x_valid = x_valid.transpose(0, 2, 3, 1)
    x_valid = x_valid.astype('float32')
    x_valid /= 255

    x_test = x_test.reshape(10000, 3, 32, 32)
    x_test = x_test.transpose(0, 2, 3, 1)
    x_test = x_test.astype('float32')
    x_test /= 255
    
    y_train = np.array(y_train)
    y_valid = np.array(y_valid)
    y_test = np.array(y_test)
    
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_valid = tf.keras.utils.to_categorical(y_valid, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    
    # preprocess data to convert from RGB -> BGR and to zero center around ImageNet dataset
    x_train = tf.keras.applications.resnet50.preprocess_input(x_train)
    x_valid = tf.keras.applications.resnet50.preprocess_input(x_valid)
    x_test = tf.keras.applications.resnet50.preprocess_input(x_test)
    
    return x_train, x_valid, x_test, y_train, y_valid, y_test, labels

In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test, labels = loadData()


# Fine Tune ResNet50 to CIFAR data

In [ ]:
# utility to display training and validation curves
def plot_metrics(metric_name, title):
    
    plt.plot(history.history[metric_name],color='blue',label='training_' + metric_name)
    plt.plot(history.history['val_' + metric_name],color='green',label='val_' + metric_name)

    plt.xlabel("epochs")
    plt.legend()
    plt.title(title)
    plt.show()

In [ ]:
EPOCHS = 50
model = tf.keras.models.load_model("~/scratch/gpfs/eysu/SoftCL/models/ResNet50+CIFAR_2")

history = model.fit(x_train, y_train, epochs=EPOCHS, validation_data = (x_valid, y_valid), batch_size=64)
loss, accuracy = model.evaluate(x_valid, y_valid, batch_size=64)


In [ ]:
plot_metrics("loss", "Loss curve")
plot_metrics("accuracy", "Accuracy curve")